# Estimador de masa mediana

In [40]:
import numpy as np
import pandas as pd

dirr = 'Datos\A2255_dccoutcut.csv'
D = 3.281618e24
G = 6.67259e-11
MS = 1.9891e30

In [41]:
data = pd.read_csv(dirr)
data

,ra,dec,vel,evel,dist,dvel
0,258.730125,63.857954,22451.5,10.99510,1.53,-1578.5
1,258.712174,64.010175,24621.2,5.47387,1.39,591.2
2,258.750503,64.168843,24003.5,8.65959,1.48,-26.5
3,258.692810,64.144804,23227.7,4.45424,1.34,-802.3
4,258.870550,63.892904,23958.3,8.05739,1.82,-71.7
...,...,...,...,...,...,...
147,258.771000,64.045063,24176.5,7.13205,1.52,146.5
148,258.527011,63.790837,22699.3,11.23490,1.18,-1330.7
149,258.425438,63.779506,24519.9,6.24049,1.02,489.9
150,258.405275,63.830808,25800.0,6.34327,0.90,1770.0


La masa mediana está dada por

$M_{med} = \dfrac{f_{me}}{G} \sum_{i} \sum_{j} [ (V_{zi} - V_{zj})^2 R_{\perp, ij} ]$

In [42]:
#Se toman los datos de interés
AR = data.ra* (np.pi/180)        #Radianes
DEC = data.dec *(np.pi/180)      #Radianes
V = data.vel* 1000          #m/s
N = len(AR)

Centroide y velocidad media:

$r_{av} = \dfrac{1}{N} \sum_{i=1}^{N} r_i$

In [43]:
cent = [sum(AR)/N, sum(DEC)/N]
Vm = np.mean(V)
print("Centroide: ", cent)
print("Velocidad media:", Vm)

Centroide:  [4.506470218665336, 1.118149807474332]
Velocidad media: 23987430.92105263


Posiciones relativas al centroide:

$R_i  = r_i - r_{av}$ 

In [44]:
#Definimos arreglos, después los llenamos acorde a las posiciones relativas
ARrel  = [ ]; DECrel = [ ]; Vrel = [ ]
for i in range(N):
    ARrel.append(AR[i] - cent[0])
    DECrel.append(DEC[i] - cent[1]) 
    Vrel.append( V[i] - Vm )
    


Ahora guardamos la diferencia de distancias angulares relativas

$\theta_{\perp, ij} = \theta_{\perp, i} - \theta_{\perp, j}$

In [45]:
#Diferencia de ángulos
thetas = np.zeros((N, N))
for i in range(N):
    for j in range(i+1, N):
        ARdif = ARrel[i] - ARrel[j]   
        DECdif = DECrel[i] - DECrel[j]
        #print(i, j, ARdif, DECdif)
        thetas[i, j] = np.sqrt( ARdif**2 + DECdif**2  )
        #print("(",i,",", j, ")", end='\t')
        #print(i==j, end='\t')

thetas


array([[0.        , 0.00267517, 0.00543768, ..., 0.00549122, 0.00568947,
        0.00506305],
       [0.        , 0.        , 0.00284894, ..., 0.00642284, 0.00620414,
        0.00633227],
       [0.        , 0.        , 0.        , ..., 0.00885228, 0.00843285,
        0.00891334],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.00096206,
        0.00084604],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.00175735],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [46]:
suma = []
matriz = np.zeros([N, N])

for i in range(N):
    for j in range(i+1, N):
        #print("[V[",i,"] - V[", j,"]]thetas[", i, ",", j,"]")
        qty = ((Vrel[i] - Vrel[j])**2)*thetas[i, j]
        suma.append( qty)
        matriz[i, j] = qty
Sum = sum(suma)
Sum
print(len(matriz.flatten()))
max (matriz.flatten())


23104


144015607075.93665

In [47]:
array = matriz.flatten()
array2 = []

for i in range( len(array) ):
    if array[i] > 0: array2.append(array[i])

med = np.mean(array2)

In [49]:
f = 6.5

M = f*Sum*D/G
M2 = f*med*D/G
print("Masa (kg): ", M)
print("Masa2 (kg): ", M2)


print("En masas solares")
print("Masa:" , M/MS, "M⊙")
print("Masa2: {:.2E}".format(M2/MS))



Masa (kg):  4.5564987314629007e+49
Masa2 (kg):  3.9708049947389017e+45
En masas solares
Masa: 2.2907338652973208e+19 M⊙
Masa2: 2.00E+15
